In [1]:
!pip install pandas 
!pip install mysql-connector-python
!pip install sqlalchemy
!pip install python-dotenv



[notice] A new release of pip available: 22.3 -> 24.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3 -> 24.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3 -> 24.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3 -> 24.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine, types
import mysql.connector
from mysql.connector import Error


In [3]:
def df_to_sql(df, table_name, connection_string):
    try:
        # Crear un motor de conexión a la base de datos
        engine = create_engine(connection_string)

        # Añadir una columna de índice como clave primaria si no existe
        if 'id' not in df.columns:
            df.insert(0, 'id', range(1, 1 + len(df)))

        # Obtener el tipo de datos de cada columna en el DataFrame
        dtype_dict = {col: types.VARCHAR(length=255) for col in df.columns}
        dtype_dict['id'] = types.INTEGER().with_variant(types.BIGINT(), 'sqlite')
        
        df.to_sql(name=table_name, con=engine, if_exists='replace', index=False, dtype=dtype_dict)
        # print(f"✅ Datos insertados en la tabla {table_name} exitosamente.")
        
    except Exception as e:
        # print(f"❌ Error al insertar datos en MySQL: {e}")
        pass


In [4]:
try:
    df = pd.read_csv('../data/MunE19_c.csv', sep=';', encoding='ISO-8859-1')
    print(df.head())
except Exception as e:
    print(f"❌ Error al leer el archivo CSV: {e}")

            TH  Cod Comarca  Cod Municipio               Ámbito   Censo  \
0  Araba-Alava            1             59      VITORIA-GASTEIZ  190184   
1  Araba-Alava            2              1     ALEGRÍA-DULANTZI    2016   
2  Araba-Alava            2              3              ARAMAIO    1176   
3  Araba-Alava            2              6              ARMIÑÓN     176   
4  Araba-Alava            2              8  ARRATZUA-UBARRUNDIA     803   

   Votantes  Nulos  Válidos  Blancos  Votos Candidatura  ...  CPE  MIEL  IZQP  \
0    119287  798.0   118489    908.0             117581  ...  126    49    57   
1      1326   15.0     1311     25.0               1286  ...    1     1     0   
2       835    2.0      833      4.0                829  ...    0     0     0   
3       124    2.0      122      3.0                119  ...    1     0     0   
4       531    2.0      529      4.0                525  ...    0     0     0   

   PCPE-PCPC-PCPA  PODEMOS-IU  JUNTS  IGRE  VOLT  CV-EC  CEX-C

In [5]:
# Cargar variables de entorno desde el archivo .env
load_dotenv()

# Obtener las variables de entorno
host = os.getenv('HOST')
port = os.getenv('PORT')
database = os.getenv('DATABASE')
user = os.getenv('USER')
password = os.getenv('PASSWORD')

In [6]:
try:
    # Conectar a MySQL
    conn = mysql.connector.connect(
        host=host,
        port=port,
        database=database,
        user=user,
        password=password
    )

    if conn.is_connected():
        print('✅ Conexión establecida a MySQL')
        cursor = conn.cursor()

        # Añadir una columna de índice como clave primaria si no existe
        if 'id' not in df.columns:
            df.insert(0, 'id', range(1, 1 + len(df)))

        # Definir la cadena de conexión con los detalles correctos
        connection_string = f'mysql+mysqlconnector://{user}:{password}@{host}:{port}/{database}'

        # Llamar a la función para convertir el DataFrame a SQL
        df_to_sql(df, 'elecc_europa_2019_municipios', connection_string)

    else:
        print('❌ No se pudo conectar a MySQL')

except Error as e:
    print(f"❌ Error al conectar a MySQL: {e}")

finally:
    if 'conn' in locals() and conn.is_connected():
        cursor.close()
        conn.close()
        print('🔒 Conexión cerrada.')


✅ Conexión establecida a MySQL
🔒 Conexión cerrada.
